In [ ]:
# Imports
from src import *
import scipy.sparse.linalg as spsl
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
# Hamiltonian Parameters
num_qubits = 15
h = 1
J = 1

# Algorithm Parameters
max_energy_level = 4
kd_ratio = 2.5
noise_threshold = 1e-2
epsilon = 1e-3
K_values = list(range(0,500,1))
delta_t = 0.083
num_modmd_observables = 6
num_trials = 20

In [ ]:
# Construct Hamiltonian, get true eigenergies
xx_strings = ['I' * i + 'ZZ' + 'I'* (num_qubits - i - 2) for i in range(num_qubits-1)]
z_strings = ['I' * i + 'X' + 'I'* (num_qubits - i - 1) for i in range(num_qubits)]

sparse_tfim_hamiltonian = SparsePauliOp.from_list([(s,-J) for s in xx_strings] + [(s,-h) for s in z_strings]).to_matrix(sparse=True)
true_eigenenergies = sorted(spsl.eigsh(sparse_tfim_hamiltonian,k=max_energy_level+1,which = 'SA')[0])

In [ ]:
# Construct reference state
indices = np.argsort(sparse_tfim_hamiltonian.diagonal())
reference_state = bitstring_superposition_state(num_qubits,[bin(indices[i])[2:] for i in range(6)])

# Get evolved reference states
max_K = K_values[-1]
max_d = int(max_K/kd_ratio)
time_evolution_operator = -1j*sparse_tfim_hamiltonian*delta_t
evolved_reference_states = spsl.expm_multiply(time_evolution_operator,reference_state,start=0,stop=max_d+max_K+1,num = max_d+max_K+2)

In [ ]:
# # ODMD Results
odmd_observables = [SparsePauliOp('I' * num_qubits).to_matrix(sparse=True)]
odmd_results = []

X_elements = generate_X_elements(odmd_observables,max_d,max_K,reference_state,evolved_reference_states)

for trial in range(num_trials):
    
    gaussian_noise = np.random.normal(0,epsilon,size=X_elements.shape) + 1j * np.random.normal(0,epsilon,size=X_elements.shape)
    noisy_X_elements = X_elements + gaussian_noise

    odmd_results.append(varying_K_results(len(odmd_observables),noise_threshold,noisy_X_elements,delta_t,K_values,kd_ratio,max_energy_level))

In [ ]:
# MODMD Results
modmd_results = []

for trial in range(num_trials):
    
    modmd_observables = random_one_local_paulis(num_qubits,num_modmd_observables)

    X_elements = generate_X_elements(modmd_observables,max_d,max_K,reference_state,evolved_reference_states)
    gaussian_noise = np.random.normal(0,epsilon,size=X_elements.shape) + 1j * np.random.normal(0,epsilon,size=X_elements.shape)
    noisy_X_elements = X_elements + gaussian_noise

    modmd_results.append(varying_K_results(len(modmd_observables),noise_threshold,noisy_X_elements,delta_t,K_values,kd_ratio,max_energy_level))

In [ ]:
# Compute errors
absolute_odmd_errors = np.array([np.abs(odmd_results[i] - true_eigenenergies) for i in range(num_trials)])
absolute_modmd_errors = np.array([np.abs(modmd_results[i] - true_eigenenergies) for i in range(num_trials)[0:17]])

In [ ]:
# Plotting
energy_levels = [0,1,2,3]
algorithm = 'MODMD'
colors = ['#c8f0a5','#7bccc4','#43a2ca','#0868ac']
labels = [r'$|\delta E_0|$',r'$|\delta E_1|$',r'$|\delta E_2|$',r'$|\delta E_3|$']

for i, energy_level in enumerate(energy_levels):

    if algorithm == 'ODMD':
        odmd_average = np.average(absolute_odmd_errors,0)[:,energy_level]
        odmd_std = np.std(absolute_odmd_errors,0)[:,energy_level]
        plt.semilogy(K_values, odmd_average,'--',color = colors[i],label = labels[i])
        plt.fill_between(K_values,odmd_average,odmd_average + odmd_std,color = colors[i],alpha = .3)
        
    elif algorithm == 'MODMD':
        modmd_average = np.average(absolute_modmd_errors,0)[:,energy_level]
        modmd_std = np.std(absolute_modmd_errors,0)[:,energy_level]
        plt.semilogy(K_values,modmd_average,color = colors[i],label = labels[i])
        plt.fill_between(K_values,modmd_average,modmd_average + modmd_std,color = colors[i],alpha = .3)

plt.axhline(epsilon, color = '#95ab9b', linestyle = '--', label = r'$\epsilon_\text{noise}$')
matplotlib.rcParams.update({'font.size': 14})
plt.xlabel(r'K ($\propto \text{Simulation Time})$')
plt.ylabel(r'Absolute Error')
plt.legend(fontsize=14)